In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/z-am-expert-av2019/campaign_data.csv
/kaggle/input/z-am-expert-av2019/test_QyjYwdj.csv
/kaggle/input/z-am-expert-av2019/coupon_item_mapping.csv
/kaggle/input/z-am-expert-av2019/item_data.csv
/kaggle/input/z-am-expert-av2019/train.csv
/kaggle/input/z-am-expert-av2019/customer_transaction_data.csv
/kaggle/input/z-am-expert-av2019/customer_demographics.csv
/kaggle/input/z-am-expert-av2019/sample_submission_Byiv0dS.csv


In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
import matplotlib.pyplot as plt, seaborn as sns
from sklearn import preprocessing, metrics
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.sparse import csr_matrix, coo_matrix
from scipy.sparse.linalg import svds
import time
import os
import gc
gc.collect()
pd.options.display.max_columns=1000
%matplotlib inline

In [3]:
def df_profile(data_frame = None):
    #import pandas as pd, numpy as np
    #read the csv file
    d = []
    for column in data_frame.columns:
        d.append({'column_name': column,
                  'data_type': data_frame[column].dtype,
                  'total_cnt': data_frame[column].size,
                  'not_null_col_cnt': data_frame[column].count(),
                  'null_count': sum(pd.isnull(data_frame[column])),
                  'unique_count': data_frame[column].nunique(),
                  'null_by_total_cnt': sum(pd.isnull(data_frame[column]))/data_frame[column].size,
                  'unique_by_not_null_cnt': data_frame[column].nunique()/data_frame[column].count(),
                  'unique_by_total_cnt': data_frame[column].nunique()/data_frame[column].size
                 })
        
    #d 
    
    df = pd.DataFrame(d,columns=d[0].keys())
    
    return df

def z_labenc(train = None, test=None, cat_cols=None):
    cat_cols=cat_cols
    for col in cat_cols:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
        train[col] = lbl.transform(list(train[col].values.astype('str')))
        test[col] = lbl.transform(list(test[col].values.astype('str')))
        print(col)

################################################
##Calculate RMSE
def rmse(x,y): return np.sqrt(((x-y)**2).mean())

################################################
##Plot LGBM Feature Importances
def plot_lgb_imp(trainX=None,model=None,n=10,fs=(10,8)):
    sns.set(style="whitegrid")
    
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(model.feature_name(), model.feature_importance()):
        feats[feature] = importance #add the name/value pair
    importances = pd.DataFrame.from_dict(feats, orient='index')
    
    importances.columns = ["imp"]
    importances = importances.sort_values(by='imp').tail(n)
    
    fig, ax = plt.subplots(figsize=fs)
    y_pos = np.arange(len(importances))
    ax.barh(y_pos, importances['imp'], color='blue', ecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(importances.index)
    plt.show()
    
    importances = importances.sort_values(by='imp',ascending=False).tail(n)
    
    return importances

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def plotly_bar(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.values[::-1],
        x=cnt_srs.index[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

In [4]:
def get_val_cnt(df = None, col_name = None, fs=(8,4), sort_idx=False):
    val_cnt = df[col_name].value_counts()
    
    #useful for date columns where we need data sorted according to date
    if sort_idx:
        val_cnt = df[col_name].value_counts().sort_index()
    
    print(f"Unique {col_name} count:")
    print(val_cnt)
    
    plt.figure(figsize=fs)
    sns.barplot(val_cnt.index, val_cnt.values, alpha=0.8)
    plt.ylabel('Count', fontsize=12)
    plt.xlabel(col_name, fontsize=12)
    plt.xticks(rotation='vertical')
    plt.show()

def plot_with_hue(df = None, plot_col_name = None, hue_col_name = None, fs=(8,4),df_name = "DataFrame"):
    plt.figure(figsize=fs)
    sns.countplot(data=df, x=plot_col_name, hue=hue_col_name, alpha=0.8)
    plt.xticks(rotation='vertical')
    plt.xlabel(plot_col_name)
    plt.ylabel("Count")
    plt.title(plot_col_name + " in "+ df_name)
    plt.show()

def get_common_cnt(df1,df2,col_name):
    return len(set(df1[col_name]).intersection(set(df2[col_name])))

def df_profile(data_frame = None):
    #import pandas as pd, numpy as np
    #read the csv file
    d = []
    for column in data_frame.columns:
        d.append({'column_name': column,
                  'data_type': data_frame[column].dtype,
                  'total_cnt': data_frame[column].size,
                  'not_null_col_cnt': data_frame[column].count(),
                  'null_count': sum(pd.isnull(data_frame[column])),
                  'unique_count': data_frame[column].nunique(),
                  'null_by_total_cnt': sum(pd.isnull(data_frame[column]))/data_frame[column].size,
                  'unique_by_not_null_cnt': data_frame[column].nunique()/data_frame[column].count(),
                  'unique_by_total_cnt': data_frame[column].nunique()/data_frame[column].size
                 })
        
    #d 
    
    df = pd.DataFrame(d,columns=d[0].keys())
    
    return df

def train_test_cat_dist(train_df,test_df,col_list):
    #used for distribution comparison of categorical variable between train and test
    #col_list = list of column whose distribution needs to be displayed
    #calls get_val_cnt
    for colname in col_list:
        print(colname)
        print("---------------------")
        print("Training Set")
        get_val_cnt(train_df,colname)
        print("Test Set")
        get_val_cnt(test_df,colname)
        print("")
        print("======================")

In [5]:
path='/kaggle/input/z-am-expert-av2019/'
TARGET_VAR='redemption_status'

In [6]:
%%time
train = pd.read_csv(path+'train.csv')
campaign= pd.read_csv(path+'campaign_data.csv')
coupon_item= pd.read_csv(path+'coupon_item_mapping.csv')
customer_demographics= pd.read_csv(path+'customer_demographics.csv')
customer_transaction= pd.read_csv(path+'customer_transaction_data.csv',parse_dates=['date'])
item= pd.read_csv(path+'item_data.csv')
test = pd.read_csv(path+'test_QyjYwdj.csv')


CPU times: user 1.25 s, sys: 204 ms, total: 1.45 s
Wall time: 1.47 s


In [7]:
train.name="train"
campaign.name="campaign"
coupon_item.name="coupon_item"
customer_demographics.name="customer_demographics"
customer_transaction.name="customer_transaction"
item.name="item"
test.name="test"

In [8]:
train_df=train.copy()
test_df=test.copy()

### Create transaction aggregate from customer transaction

In [9]:
customer_transaction["trans_month"] = customer_transaction.date.dt.month
customer_transaction["trans_week_of_yr"] = customer_transaction.date.dt.weekofyear
customer_transaction["trans_yr"] = customer_transaction.date.dt.year

In [10]:
item.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [11]:
%%time
#Not all items may have coupon. Create a flag to indicate which items have coupons
item["is_couponed"] = item["item_id"].isin(coupon_item["item_id"].unique()).astype(np.int8)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.81 ms


In [12]:
item.head()

,item_id,brand,brand_type,category,is_couponed
0,1,1,Established,Grocery,1
1,2,1,Established,Miscellaneous,0
2,3,56,Local,Bakery,0
3,4,56,Local,Grocery,1
4,5,56,Local,Grocery,0


In [13]:
def calc_redeem(row):
    if (row["is_couponed"] == 1) & (row["coupon_discount"] == 0): return 0
    if (row["is_couponed"] == 1) & (row["coupon_discount"] < 0): return 1
    if (row["is_couponed"] == 0): return -99


In [14]:
%%time
trans_calc_df = pd.DataFrame()
for camp_id in campaign["campaign_id"].unique():
    print(f"campaign_id={camp_id}")

    target_date = campaign.loc[campaign["campaign_id"] == camp_id]["start_date"].values[0]
    trans_df = customer_transaction.loc[customer_transaction["date"] < target_date,:]
    
    print(trans_df.shape)
    trans_df = pd.merge(trans_df,item,how="left",on="item_id")
    print(trans_df.shape)
    
    trans_df[TARGET_VAR] = trans_df.apply(lambda row: calc_redeem(row), axis=1)
    
    ############
    cust_redeem_prof = trans_df.groupby(["customer_id","redemption_status"])["date"].count().reset_index()
    cust_redeem_prof.columns = ['customer_id', 'redemption_status', 'count']
    
    cust_redeem_prof = cust_redeem_prof.pivot_table(values='count',index=['customer_id'],columns='redemption_status',fill_value=0).reset_index()
    cust_redeem_prof.columns = ["customer_id","cust_non_coupon_trans_count","cust_non_redeem_trans_count","cust_redeem_trans_count"]
    
    cust_redeem_prof["cust_coupon_trans_count"] = cust_redeem_prof["cust_non_redeem_trans_count"]+cust_redeem_prof["cust_redeem_trans_count"]
    cust_redeem_prof["cust_total_trans_count"] = cust_redeem_prof["cust_non_coupon_trans_count"]+cust_redeem_prof["cust_coupon_trans_count"]
    cust_redeem_prof["cust_redeem_nonredeem_ratio"] = cust_redeem_prof["cust_redeem_trans_count"]/cust_redeem_prof["cust_non_redeem_trans_count"]
    cust_redeem_prof["cust_redeem_rate"] = cust_redeem_prof["cust_redeem_trans_count"]/cust_redeem_prof["cust_coupon_trans_count"]
    cust_redeem_prof["cust_coupon_noncoupon_ratio"] =cust_redeem_prof["cust_coupon_trans_count"]/cust_redeem_prof["cust_non_coupon_trans_count"]
    cust_redeem_prof["cust_coupon_trans_perc"] =cust_redeem_prof["cust_coupon_trans_count"]/cust_redeem_prof["cust_total_trans_count"]
    #############
    
    #Getting only couponed ransactions for calculating transagg
    trans_df =  trans_df.loc[trans_df["is_couponed"] == 1,:]
    
    trans_agg = trans_df.groupby('customer_id').agg(
                                     {
                                      'item_id':['nunique','count'],
                                      'quantity':['mean','max','min','std','sum'],
                                      'selling_price':['mean','max','min','std','sum'],
                                      'other_discount':['mean','max','min','std','sum'],
                                      'coupon_discount':['mean','max','min','std','sum']
                                     }).reset_index()
    
    trans_agg.columns = ['customer_id',
                         'cust_item_uniq_count','cust_item_total_count',
                         'cust_quantity_mean','cust_quantity_max','cust_quantity_min','cust_quantity_std','cust_quantity_sum',
                         'cust_sp_mean','cust_sp_max','cust_sp_min','cust_sp_std','cust_sp_sum',
                         'cust_other_discount_mean','cust_other_discount_max','cust_other_discount_min','cust_other_discount_std','cust_other_discount_sum',
                         'cust_coupon_discount_mean','cust_coupon_discount_max','cust_coupon_discount_min','cust_coupon_discount_std','cust_coupon_discount_sum'
                        ]
    
    trans_agg = pd.merge(trans_agg,cust_redeem_prof,how="left",on="customer_id")
    
    trans_agg["campaign_id"] = camp_id

    trans_calc_df = pd.concat([trans_calc_df,trans_agg])
    

campaign_id=24
(1324566, 10)
(1324566, 14)
campaign_id=25
(1324566, 10)
(1324566, 14)
campaign_id=20
(1324566, 10)
(1324566, 14)
campaign_id=23
(1324566, 10)
(1324566, 14)
campaign_id=21
(1324566, 10)
(1324566, 14)
campaign_id=22
(1324566, 10)
(1324566, 14)
campaign_id=18
(1324566, 10)
(1324566, 14)
campaign_id=19
(1324566, 10)
(1324566, 14)
campaign_id=17
(1324566, 10)
(1324566, 14)
campaign_id=16
(1324566, 10)
(1324566, 14)
campaign_id=13
(1203485, 10)
(1203485, 14)
campaign_id=11
(1130331, 10)
(1130331, 14)
campaign_id=12
(1130331, 10)
(1130331, 14)
campaign_id=10
(1324566, 10)
(1324566, 14)
campaign_id=9
(1324566, 10)
(1324566, 14)
campaign_id=8
(952011, 10)
(952011, 14)
campaign_id=7
(912057, 10)
(912057, 14)
campaign_id=6
(898658, 10)
(898658, 14)
campaign_id=3
(794938, 10)
(794938, 14)
campaign_id=5
(1324566, 10)
(1324566, 14)
campaign_id=4
(1318189, 10)
(1318189, 14)
campaign_id=1
(765157, 10)
(765157, 14)
campaign_id=2
(780429, 10)
(780429, 14)
campaign_id=30
(701346, 10)
(701

In [15]:
trans_calc_df.shape

(44278, 33)

In [16]:
trans_calc_df.head()

,customer_id,cust_item_uniq_count,cust_item_total_count,cust_quantity_mean,cust_quantity_max,cust_quantity_min,cust_quantity_std,cust_quantity_sum,cust_sp_mean,cust_sp_max,cust_sp_min,cust_sp_std,cust_sp_sum,cust_other_discount_mean,cust_other_discount_max,cust_other_discount_min,cust_other_discount_std,cust_other_discount_sum,cust_coupon_discount_mean,cust_coupon_discount_max,cust_coupon_discount_min,cust_coupon_discount_std,cust_coupon_discount_sum,cust_non_coupon_trans_count,cust_non_redeem_trans_count,cust_redeem_trans_count,cust_coupon_trans_count,cust_total_trans_count,cust_redeem_nonredeem_ratio,cust_redeem_rate,cust_coupon_noncoupon_ratio,cust_coupon_trans_perc,campaign_id
0,1,211,507,1.130178,5,1,0.464997,573,103.824063,445.25,21.37,58.993142,52638.80,-23.482899,0.0,-372.23,42.386221,-11905.83,-2.349980,0.0,-89.05,9.063248,-1191.44,541,458,49,507,1048,0.106987,0.096647,0.937153,0.483779,24
1,2,216,268,1.123134,4,1,0.419276,301,105.014851,640.80,17.45,71.657352,28143.98,-21.074067,0.0,-312.03,40.504439,-5647.85,-0.797463,0.0,-89.05,7.969548,-213.72,151,265,3,268,419,0.011321,0.011194,1.774834,0.639618,24
2,3,223,394,1.713198,16,1,1.510431,675,113.724137,862.00,8.90,87.584942,44807.31,-26.110812,0.0,-463.06,43.679586,-10287.66,-2.458071,0.0,-71.24,10.091462,-968.48,311,366,28,394,705,0.076503,0.071066,1.266881,0.558865,24
3,4,74,129,1.317829,5,1,0.790416,170,141.830388,623.35,34.55,98.434163,18296.12,-20.098760,0.0,-390.04,45.269237,-2592.74,-0.690310,0.0,-89.05,7.840415,-89.05,91,128,1,129,220,0.007812,0.007752,1.417582,0.586364,24
4,5,289,499,1.438878,24,1,1.718600,718,122.223667,605.18,13.89,92.585881,60989.61,-14.755571,0.0,-216.21,29.878654,-7363.03,-0.182024,0.0,-71.24,3.305822,-90.83,293,497,2,499,792,0.004024,0.004008,1.703072,0.630051,24


In [17]:
trans_calc_df.columns

Index(['customer_id', 'cust_item_uniq_count', 'cust_item_total_count',
       'cust_quantity_mean', 'cust_quantity_max', 'cust_quantity_min',
       'cust_quantity_std', 'cust_quantity_sum', 'cust_sp_mean', 'cust_sp_max',
       'cust_sp_min', 'cust_sp_std', 'cust_sp_sum', 'cust_other_discount_mean',
       'cust_other_discount_max', 'cust_other_discount_min',
       'cust_other_discount_std', 'cust_other_discount_sum',
       'cust_coupon_discount_mean', 'cust_coupon_discount_max',
       'cust_coupon_discount_min', 'cust_coupon_discount_std',
       'cust_coupon_discount_sum', 'cust_non_coupon_trans_count',
       'cust_non_redeem_trans_count', 'cust_redeem_trans_count',
       'cust_coupon_trans_count', 'cust_total_trans_count',
       'cust_redeem_nonredeem_ratio', 'cust_redeem_rate',
       'cust_coupon_noncoupon_ratio', 'cust_coupon_trans_perc', 'campaign_id'],
      dtype='object')

In [18]:
trans_calc_df.loc[trans_calc_df["customer_id"]==10,:]

,customer_id,cust_item_uniq_count,cust_item_total_count,cust_quantity_mean,cust_quantity_max,cust_quantity_min,cust_quantity_std,cust_quantity_sum,cust_sp_mean,cust_sp_max,cust_sp_min,cust_sp_std,cust_sp_sum,cust_other_discount_mean,cust_other_discount_max,cust_other_discount_min,cust_other_discount_std,cust_other_discount_sum,cust_coupon_discount_mean,cust_coupon_discount_max,cust_coupon_discount_min,cust_coupon_discount_std,cust_coupon_discount_sum,cust_non_coupon_trans_count,cust_non_redeem_trans_count,cust_redeem_trans_count,cust_coupon_trans_count,cust_total_trans_count,cust_redeem_nonredeem_ratio,cust_redeem_rate,cust_coupon_noncoupon_ratio,cust_coupon_trans_perc,campaign_id
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,24
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,25
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,20
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,23
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,21
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,22
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,18
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,19
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,17
9,10,144,262,1.103053,3,1,0.316939,289,92.565344,587.02,13.54,65.824285,24252.12,-14.419237,0.0,-213.72,28.801448,-3777.84,0.0,0.0,0.0,0.0,0.0,229,262,0,262,491,0.0,0.0,1.144105,0.533605,16


In [19]:
trans_calc_df.to_csv("trans_calc_df.csv",index=False)